In [ ]:
# !pip install transformers datasets google-colab google-auth

# from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, Trainer, TrainingArguments
# from datasets import load_dataset, ClassLabel, Value, DatasetDict, Dataset
# import pandas as pd
# from google.colab import auth
# import gspread
# from oauth2client.client import GoogleCredentials

In [ ]:
#!pip install transformers accelerate

In [ ]:
#!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
# # Import necessary library
# import torch

# # Check if CUDA is available and identify the GPU
# if torch.cuda.is_available():
#     print('GPU is available:', torch.cuda.get_device_name(0))
# else:
#     print('GPU is not available, training will run on CPU.')


In [ ]:
# from google.colab import auth
# import gspread
# from google.auth import default

# # Authenticate to Google
# auth.authenticate_user()
# creds, _ = default()
# gc = gspread.authorize(creds)

# # Load the data from Google Sheets
# worksheet = gc.open('Copy of Merged Data: matched labels').sheet1
# rows = worksheet.get_all_values()
# df = pd.DataFrame(rows[1:], columns=rows[0])

# # Display the first few rows to confirm
# print(df.head())

In [ ]:
# # Concatenate the caption and question
# df['text'] = df['Caption'] + " [SEP] " + df['Question']

# # Convert labels to integers
# df['labels'] = df['Label_gemini'].astype(int)

# # Create a Hugging Face dataset from the DataFrame
# dataset = Dataset.from_pandas(df[['text', 'labels']])
# dataset = DatasetDict({'train': dataset})

# # Check the dataset
# print(dataset['train'][0])

In [ ]:
# tokenizer = RobertaTokenizerFast.from_pretrained('deepset/tinyroberta-squad2')
# model = RobertaForSequenceClassification.from_pretrained('deepset/tinyroberta-squad2', num_labels=2)

# def tokenize_function(examples):
#     return tokenizer(examples['text'], padding="max_length", truncation=True)

# # Tokenize the data
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=3,              # number of training epochs
#     per_device_train_batch_size=16,  # batch size for training
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets['train'],
# )

# # Train the model
# trainer.train()


In [ ]:
# # Save the model
# model_path = "path_to_save_model"
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

# # Evaluate the model
# results = trainer.evaluate()
# print(results)

In [ ]:
# Install necessary libraries
!pip install transformers datasets google-colab google-auth accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.

In [ ]:
# Import libraries
from transformers import AutoTokenizer, CanineForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, ClassLabel, Value, DatasetDict, Dataset
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default

# Authenticate and load data
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('Merged Data: matched labels').sheet1
rows = worksheet.get_all_values()
df = pd.DataFrame(rows[1:], columns=rows[0])

# Concatenate caption and question, convert labels
df['text'] = df['Caption'] + " [SEP] " + df['Question']
df['labels'] = df['Label_gemini'].astype(int)


In [ ]:
import numpy as np
import torch

In [ ]:
# Split data into train and validation sets
train_df = df.sample(frac=0.8, random_state=42)  # 80% for training
valid_df = df.drop(train_df.index)               # Remaining 20% for validation

In [ ]:
# Create datasets from DataFrames
train_dataset = Dataset.from_pandas(train_df[['text', 'labels']])
valid_dataset = Dataset.from_pandas(valid_df[['text', 'labels']])
datasets = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset
})

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('google/canine-s')
model = CanineForSequenceClassification.from_pretrained('google/canine-s', num_labels=2)
# Move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
tokenized_datasets = datasets.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/528M [00:00<?, ?B/s]

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3648 [00:00<?, ? examples/s]

Map:   0%|          | 0/912 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,  # Adjust batch size according to GPU memory
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Define a simple accuracy metric function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': (predictions == labels).mean()}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Save the model
model_path = "/content/drive/MyDrive/transformer_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Epoch,Training Loss,Validation Loss,Accuracy
1,0.446700,0.338322,0.900219
2,0.424100,0.324728,0.900219
3,0.438800,0.324585,0.900219


{'eval_loss': 0.3383219540119171, 'eval_accuracy': 0.9002192982456141, 'eval_runtime': 13.9228, 'eval_samples_per_second': 65.504, 'eval_steps_per_second': 8.188, 'epoch': 3.0}


('/content/drive/MyDrive/transformer_model/tokenizer_config.json',
 '/content/drive/MyDrive/transformer_model/special_tokens_map.json',
 '/content/drive/MyDrive/transformer_model/added_tokens.json')

In [ ]:
from sklearn.metrics import f1_score

# Use the modified TrainingArguments and compute_metrics in the Trainer setup
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,  # Adjust batch size according to GPU memory
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Using F1-score to determine the best model
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')  # Calculate weighted F1-score
    return {
        'accuracy': (predictions == labels).mean(),
        'f1': f1
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Save the model
model_path = "/content/drive/MyDrive/transformer_model_1"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.436600,0.324983,0.900219,0.852949
2,0.426600,0.324622,0.900219,0.852949
3,0.442000,0.324596,0.900219,0.852949


{'eval_loss': 0.3249830901622772, 'eval_accuracy': 0.9002192982456141, 'eval_f1': 0.8529486945870157, 'eval_runtime': 13.9422, 'eval_samples_per_second': 65.413, 'eval_steps_per_second': 8.177, 'epoch': 3.0}


('/content/drive/MyDrive/transformer_model_1/tokenizer_config.json',
 '/content/drive/MyDrive/transformer_model_1/special_tokens_map.json',
 '/content/drive/MyDrive/transformer_model_1/added_tokens.json')